(sec:integrals)=
# Integrals

## Exposure to Python layer

Several integrals are exposed to the Python layer in VeloxChem.

In [7]:
import numpy as np
import veloxchem as vlx

mol_str = """
O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

molecule = vlx.Molecule.read_str(mol_str, units="angstrom")
basis = vlx.MolecularBasis.read(molecule, "cc-pVDZ")

* Info * Reading basis set from file: /Users/panor/opt/miniconda3/envs/vlxnb/lib/python3.10/site-packages/veloxchem/basis/CC-PVDZ
                                                                                                                          
                                              Molecular Basis (Atomic Basis)                                              
                                                                                                                          
                                  Basis: CC-PVDZ                                                                          
                                                                                                                          
                                  Atom Contracted GTOs          Primitive GTOs                                            
                                                                                                                          
         

### Overlap

In [8]:
overlap_drv = vlx.OverlapIntegralsDriver()

S = overlap_drv.compute(molecule, basis).to_numpy()

### Kinetic energy

In [12]:
# one-electron Hamiltonian
kinetic_drv = vlx.KineticEnergyIntegralsDriver()

T = kinetic_drv.compute(molecule, basis).to_numpy()

### Nuclear potential

In [13]:
nucpot_drv = vlx.NuclearPotentialIntegralsDriver()

V = -1.0 * nucpot_drv.compute(molecule, basis).to_numpy()

### One-electron Hamiltonian

In [14]:
h = T + V

### Electric dipole moment

In [23]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()

dipole_mats = dipole_drv.compute(molecule, basis)

mu_x = -1.0 * dipole_mats.x_to_numpy()
mu_y = -1.0 * dipole_mats.y_to_numpy()
mu_z = -1.0 * dipole_mats.z_to_numpy()

### Electron repulsion integrals

In [33]:
eri_drv = vlx.ElectronRepulsionIntegralsDriver()

g = eri_drv.compute_in_memory(molecule, basis)

print(g.shape)

(24, 24, 24, 24)


## Integral transformation

### Conventional

Most correlated wavefunction theories rely on the integrals in the molecular orbital basis. For some methods, like Møller-Plesset second order perturbation theory, this step is even the most time-consuming step of the calculation.

Conventionally, this is done by transforming the atomic orbital integrals (a four-dimensional tensor) to molecular basis. For example the $\langle ij|ab \rangle$ integrals can be obtained via

\begin{eqnarray*}
\langle \mu \nu|\lambda b \rangle &= \sum_{\sigma} C_{\sigma b} \langle \mu \nu | \lambda \sigma \rangle \\
\langle \mu \nu|a b \rangle &= \sum_{\lambda} C_{\lambda a} \langle \mu \nu | \lambda b \rangle \\
\langle \mu j|a b \rangle &= \sum_{\nu} C_{\nu j} \langle \mu \nu | a b \rangle \\
\langle i j|a b \rangle &= \sum_{\mu} C_{\mu i} \langle \mu j | a b \rangle
\end{eqnarray*}

where $\mu$, $\nu$, $\lambda$ and $\sigma$ denote the atomic orbitals.
The computational cost of this procedure is $O(N^5)$, since each summation involves five indices. Note that the intermediate result of the transformation needs to be explicitly stored in memory. This can be demanding as the required memory increases as $O(N^4)$ where $N$ is the number of contracted basis functions.

In [20]:
# get the MO coefficients

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.compute(molecule, basis)

C = scf_drv.scf_tensors["C_alpha"]

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [34]:
g_1 = np.einsum("ds, abcd -> abcs", C, g)
g_2 = np.einsum("cr, abcs -> abrs", C, g_1)
g_3 = np.einsum("bq, abrs -> aqrs", C, g_2)

g_mo = np.einsum("ap, aqrs -> pqrs", C, g_3)

print(g_mo.shape)

(24, 24, 24, 24)


Specific blocks a ERIs are made available directly in the MO basis, both in the physicist's and the chemist's notation.

In [31]:
erimo_drv = vlx.MOIntegralsDriver()

# [ov|vo]
phys_ovvo = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "phys_ovvo")

print(phys_ovvo.shape)

# (oo|vv)
chem_oovv = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_oovv")

print(chem_oovv.shape)

(5, 19, 19, 5)
(5, 5, 19, 19)


### Fock-matrix driven

An alternative way of getting the molecular orbital integrals is through the direct formation of many Fock matrices. Taking the $\langle ij|ab \rangle$ integrals as an example, we can in practice build $N_{occ} \times N_{occ}$ density matrices using the coefficients of the occupied orbitals, and form the corresponding Fock matrices that can then be transformed into molecular basis by the coefficients of the virtual orbitals. 

\begin{eqnarray*}
D^{ij}_{\mu \nu} &= C_{\mu i} C_{\nu j} \\
K^{ij}_{\lambda \sigma} &= \sum_{\mu\nu} \langle \mu \nu| \lambda \sigma \rangle D^{ij}_{\mu\nu} \\
\langle ij|ab \rangle &= \sum_{\lambda \sigma} C_{\lambda a} K^{ij}_{\lambda \sigma} C_{\sigma b}
\end{eqnarray*}

The computational cost of this approach is formally $O(N^6)$; however, in practice the cost scales between $O(N^4)$ and $O(N^5)$ due to screening of integrals in the formation of Fock matrices. An advantage of this approach is that the Fock matrices can be computed and stored on individual compute nodes, making it suitable for large-scale parallelization on HPC systems.